In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Read the CSV file
df = pd.read_csv('data_03.csv', index_col=0, parse_dates=True)

# Preprocess the data
df['평균수온'] = df.mean(axis=1)
df = df[['평균수온']]
df.loc[df['평균수온'] == 0, '평균수온'] = np.nan
df = df.dropna()

# Split the data into training and test sets
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Scale the data
scaler = MinMaxScaler()
train_data_scaled = scaler.fit_transform(train_data)
test_data_scaled = scaler.transform(test_data)

# Convert the data to PyTorch tensors
train_tensor = torch.from_numpy(train_data_scaled).float()
test_tensor = torch.from_numpy(test_data_scaled).float()

# Define the GRU model
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GRUModel, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers=1, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.gru(x)
        out = self.fc(out[:, -1, :])
        return out

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the model parameters
input_size = train_tensor.shape[1]
hidden_size = 64
output_size = 1

# Instantiate the model
model = GRUModel(input_size, hidden_size, output_size).to(device)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Convert the tensors to the appropriate device
train_tensor = train_tensor.to(device)
test_tensor = test_tensor.to(device)

# Train the model
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(train_tensor.unsqueeze(0))
    loss = criterion(outputs.squeeze(), train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch: {epoch+1}, Loss: {loss.item():.4f}')

# Set the model to evaluation mode
model.eval()

# Predict the water temperature for 12/30/2022
input_data = torch.from_numpy(scaler.transform(np.array([[16.221020]]))).float().unsqueeze(0).to(device)
prediction = model(input_data)

# Convert the prediction back to its original scale
prediction = scaler.inverse_transform(prediction.cpu().detach().numpy())

print('Predicted water temperature on 12/30/2022:', prediction.item())


c:\Users\admin\anaconda3\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([2708, 1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 100, Loss: 0.0504
Epoch: 200, Loss: 0.0504
Epoch: 300, Loss: 0.0504
Epoch: 400, Loss: 0.0504
Epoch: 500, Loss: 0.0504
Epoch: 600, Loss: 0.0504
Epoch: 700, Loss: 0.0504
Epoch: 800, Loss: 0.0504
Epoch: 900, Loss: 0.0504
Epoch: 1000, Loss: 0.0504
Predicted water temperature on 12/30/2022: 9.391682624816895


c:\Users\admin\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
